In [1]:
! pip install pandas tqdm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [9]:
!nvidia-smi

Sat Nov 22 08:34:02 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   68C    P0             225W / 300W |  81029MiB / 81920MiB |    100%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [1]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

/workspace


In [3]:
!pip install datasets==3.6.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: datasets
    Found existing installation: datasets 2.21.0
    Uninstalling datasets-2.21.0:
      Successfully uninstalled datasets-2.21.0


In [4]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm

# Step 1: Load dataset
print("🔽 Loading IWSLT2017 English–German dataset...")
dataset = load_dataset("IWSLT/iwslt2017", "iwslt2017-en-de")

# Step 2: Extract English sentences from each split
def extract_english(ds):
    """Extract English sentences from a dataset split."""
    return [ex["translation"]["en"] for ex in ds]

print("📄 Extracting English sentences from each split...")
train_en = extract_english(dataset["train"])
valid_en = extract_english(dataset["validation"])
test_en  = extract_english(dataset["test"])

# Step 3: Save each split separately
print("💾 Saving splits...")

pd.DataFrame({"text": train_en}).to_csv("iwslt2017_en_train.csv", index=False)
pd.DataFrame({"text": valid_en}).to_csv("iwslt2017_en_valid.csv", index=False)
pd.DataFrame({"text": test_en}).to_csv("iwslt2017_en_test.csv", index=False)

print(f"""
✅ Done!
Train sentences: {len(train_en):,}
Validation sentences: {len(valid_en):,}
Test sentences: {len(test_en):,}

Saved files:
 - iwslt2017_en_train.csv
 - iwslt2017_en_valid.csv
 - iwslt2017_en_test.csv
""")

# Optional preview
print("🔍 Example:")
print(pd.DataFrame({'train_example': train_en[:3]}))

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔽 Loading IWSLT2017 English–German dataset...


KeyboardInterrupt: Interrupted by user

In [2]:
import pandas as pd
import re
from tqdm import tqdm

# define punctuation map
punctuation_map = {
    ',': 'COMMA',
    '.': 'PERIOD',
    '?': 'QUESTION',
    '!': 'EXCLAMATION',
    ';': 'SEMICOLON',
    ':': 'COLON',
    '-': 'HYPHEN',
    '–': 'EN_DASH',
    '—': 'EM_DASH',
    '(': 'LEFT_PAREN',
    ')': 'RIGHT_PAREN',
    '[': 'LEFT_BRACKET',
    ']': 'RIGHT_BRACKET',
    '{': 'LEFT_BRACE',
    '}': 'RIGHT_BRACE',
    '"': 'DOUBLE_QUOTE',
    "'": 'SINGLE_QUOTE',
    '…': 'ELLIPSIS',
    '/': 'SLASH',
    '\\': 'BACKSLASH',
    '@': 'AT_SYMBOL',
    '#': 'HASH',
    '$': 'DOLLAR',
    '%': 'PERCENT',
    '&': 'AMPERSAND',
    '*': 'ASTERISK',
    '+': 'PLUS',
    '=': 'EQUALS',
    '<': 'LESS_THAN',
    '>': 'GREATER_THAN',
    '|': 'PIPE',
    '^': 'CARET',
    '`': 'BACKTICK',
    '~': 'TILDE'
}

def create_token_labels(sentence):
    tokens = []
    labels = []
    # split on space but keep punctuation info
    parts = re.findall(r"\w+|[^\w\s]", sentence)
    for i, part in enumerate(parts):
        if re.match(r"\w+", part):  # token
            tokens.append(part)
            # check next item
            if i+1 < len(parts) and parts[i+1] in punctuation_map:
                labels.append(punctuation_map[parts[i+1]])
            else:
                labels.append("O")
    return tokens, labels

# preview one
ex = "What we see we believe, what we hear we register."
tokens, labels = create_token_labels(ex)
print(list(zip(tokens, labels)))

[('What', 'O'), ('we', 'O'), ('see', 'O'), ('we', 'O'), ('believe', 'COMMA'), ('what', 'O'), ('we', 'O'), ('hear', 'O'), ('we', 'O'), ('register', 'PERIOD')]


In [3]:
import re
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import BertTokenizerFast
import os
from tqdm import tqdm

# 1. define punctuation map
punctuation_map = {
    ',': 'COMMA',
    '.': 'PERIOD',
    '?': 'QUESTION',
    '!': 'EXCLAMATION',
    ';': 'SEMICOLON',
    ':': 'COLON',
    '-': 'HYPHEN',
    '–': 'EN_DASH',
    '—': 'EM_DASH',
    '(': 'LEFT_PAREN',
    ')': 'RIGHT_PAREN',
    '[': 'LEFT_BRACKET',
    ']': 'RIGHT_BRACKET',
    '{': 'LEFT_BRACE',
    '}': 'RIGHT_BRACE',
    '"': 'DOUBLE_QUOTE',
    "'": 'SINGLE_QUOTE',
    '…': 'ELLIPSIS',
    '/': 'SLASH',
    '\\': 'BACKSLASH',
    '@': 'AT_SYMBOL',
    '#': 'HASH',
    '$': 'DOLLAR',
    '%': 'PERCENT',
    '&': 'AMPERSAND',
    '*': 'ASTERISK',
    '+': 'PLUS',
    '=': 'EQUALS',
    '<': 'LESS_THAN',
    '>': 'GREATER_THAN',
    '|': 'PIPE',
    '^': 'CARET',
    '`': 'BACKTICK',
    '~': 'TILDE'
}

# Automatically create label_list from punctuation_map
label_list = ["O"] + list(punctuation_map.values())
label_to_id = {l: i for i, l in enumerate(label_list)}

print("Label list:", label_list)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Label list: ['O', 'COMMA', 'PERIOD', 'QUESTION', 'EXCLAMATION', 'SEMICOLON', 'COLON', 'HYPHEN', 'EN_DASH', 'EM_DASH', 'LEFT_PAREN', 'RIGHT_PAREN', 'LEFT_BRACKET', 'RIGHT_BRACKET', 'LEFT_BRACE', 'RIGHT_BRACE', 'DOUBLE_QUOTE', 'SINGLE_QUOTE', 'ELLIPSIS', 'SLASH', 'BACKSLASH', 'AT_SYMBOL', 'HASH', 'DOLLAR', 'PERCENT', 'AMPERSAND', 'ASTERISK', 'PLUS', 'EQUALS', 'LESS_THAN', 'GREATER_THAN', 'PIPE', 'CARET', 'BACKTICK', 'TILDE']


In [13]:
# -------------------------------
# 2. Token-label creation function
# -------------------------------
def create_token_labels(sentence):
    tokens = []
    labels = []
    parts = re.findall(r"\w+|[^\w\s]", sentence)
    for i, part in enumerate(parts):
        if re.match(r"\w+", part):  # token
            tokens.append(part)
            if i+1 < len(parts) and parts[i+1] in punctuation_map:
                labels.append(punctuation_map[parts[i+1]])
            else:
                labels.append("O")
    return tokens, labels

# -------------------------------
# 3. Load CSV and create token-label dataset
# -------------------------------
def load_and_process(csv_file):
    df = pd.read_csv(csv_file)
    all_tokens = []
    all_labels = []

    for sent in df['text']:
        tokens, labels = create_token_labels(str(sent))
        all_tokens.append(tokens)
        all_labels.append([label_to_id[l] for l in labels])

    return Dataset.from_dict({"tokens": all_tokens, "labels": all_labels})

print("📄 Processing train/val/test datasets...")
train_dataset = load_and_process("iwslt2017_en_train.csv")
val_dataset   = load_and_process("iwslt2017_en_valid.csv")
test_dataset  = load_and_process("iwslt2017_en_test.csv")

# -------------------------------
# 4. Tokenize with BERT tokenizer
# -------------------------------
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(batch["tokens"], is_split_into_words=True, truncation=True, padding="max_length", max_length=128)
    new_labels = []
    for i, label in enumerate(batch["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        for word_id in word_ids:
            if word_id is None:
                aligned_labels.append(-100)
            else:
                aligned_labels.append(label[word_id])
        new_labels.append(aligned_labels)
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
val_dataset   = val_dataset.map(tokenize_and_align_labels, batched=True)
test_dataset  = test_dataset.map(tokenize_and_align_labels, batched=True)

# Remove original columns
train_dataset = train_dataset.remove_columns(["tokens"])
val_dataset   = val_dataset.remove_columns(["tokens"])
test_dataset  = test_dataset.remove_columns(["tokens"])

# -------------------------------
# 5. Set format for PyTorch
# -------------------------------
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

# -------------------------------
# 6. Save processed datasets
# -------------------------------
os.makedirs("processed_datasets", exist_ok=True)
train_dataset.save_to_disk("processed_datasets/train")
val_dataset.save_to_disk("processed_datasets/val")
test_dataset.save_to_disk("processed_datasets/test")

print("✅ Processed datasets saved under 'processed_datasets/'")
print("You can load them later using `Dataset.load_from_disk()`")

Label list: ['O', 'COMMA', 'PERIOD', 'QUESTION', 'EXCLAMATION', 'SEMICOLON', 'COLON', 'HYPHEN', 'EN_DASH', 'EM_DASH', 'LEFT_PAREN', 'RIGHT_PAREN', 'LEFT_BRACKET', 'RIGHT_BRACKET', 'LEFT_BRACE', 'RIGHT_BRACE', 'DOUBLE_QUOTE', 'SINGLE_QUOTE', 'ELLIPSIS', 'SLASH', 'BACKSLASH', 'AT_SYMBOL', 'HASH', 'DOLLAR', 'PERCENT', 'AMPERSAND', 'ASTERISK', 'PLUS', 'EQUALS', 'LESS_THAN', 'GREATER_THAN', 'PIPE', 'CARET', 'BACKTICK', 'TILDE']
📄 Processing train/val/test datasets...


Saving the dataset (1/1 shards): 100%|██████████████████████████████████████████████████████████| 8079/8079 [00:00<00:00, 279989.94 examples/s]

✅ Processed datasets saved under 'processed_datasets/'
You can load them later using `Dataset.load_from_disk()`


In [4]:
from datasets import load_from_disk

train_dataset = load_from_disk("../processed_datasets/train")
val_dataset   = load_from_disk("../processed_datasets/val")
test_dataset  = load_from_disk("../processed_datasets/test")

In [5]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

206112
888
8079


In [ ]:
import numpy as np
import torch
from datasets import load_from_disk
from transformers import BertForTokenClassification, BertTokenizerFast, Trainer, TrainingArguments
from seqeval.metrics import f1_score, precision_score, recall_score

# -------------------------------
# 1. Device setup
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------------------
# 2. Labels
# -------------------------------
num_labels = len(label_list)
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}

# -------------------------------
# 3. Load tokenizer and model
# -------------------------------
tokenizer = BertTokenizerFast.from_pretrained("bert-large-uncased")
model = BertForTokenClassification.from_pretrained(
    "bert-large-uncased",
    num_labels=num_labels
)
model.to(device)  # explicitly move model to GPU

# -------------------------------
# 4. Metrics function
# -------------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    true_labels = []
    pred_labels = []
    
    for label_row, pred_row in zip(labels, predictions):
        true_row = []
        pred_row_labels = []
        for l, p in zip(label_row, pred_row):
            if l != -100:
                true_row.append(label_list[l])
                pred_row_labels.append(label_list[p])
        true_labels.append(true_row)
        pred_labels.append(pred_row_labels)
    
    f1 = f1_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    
    return {"f1": f1, "precision": precision, "recall": recall}

# -------------------------------
# 5. Training arguments
# -------------------------------
training_args = TrainingArguments(
    output_dir="./bert_punct_model",
    eval_steps=500,
    eval_strategy="steps",
    logging_steps=200,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available()  # mixed precision if GPU available
)

# -------------------------------
# 6. Trainer
# -------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# -------------------------------
# 7. Train
# -------------------------------
trainer.train()

# -------------------------------
# 8. Save final model
# -------------------------------
trainer.save_model("./bert_punct_model_final1")
tokenizer.save_pretrained("./bert_punct_model_final1")

In [ ]:
trainer.push_to_hub("thenlpresearcher/bert-punctuation-restoration-kaustubh")

In [27]:
# -------------------------------
# Evaluate on test set
# -------------------------------
test_metrics = trainer.evaluate(eval_dataset=test_dataset)
print("✅ Test set evaluation:")
print(f"F1 Score: {test_metrics['eval_f1']:.4f}")
print(f"Precision: {test_metrics['eval_precision']:.4f}")
print(f"Recall: {test_metrics['eval_recall']:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:271: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  arr = np.array(obj)
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: COMMA seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PERIOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SINGLE_QUOTE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: COLON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: HYPHEN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_l

✅ Test set evaluation:
F1 Score: 0.8677
Precision: 0.8717
Recall: 0.8637


In [8]:
import numpy as np
from seqeval.metrics import classification_report

# Get predictions
predictions, labels, _ = trainer.predict(test_dataset)
pred_ids = np.argmax(predictions, axis=-1)

true_labels_list = []
pred_labels_list = []

for label_row, pred_row in zip(labels, pred_ids):
    true_row = []
    pred_row_labels = []
    for l, p in zip(label_row, pred_row):
        if l != -100:  # ignore padding
            true_row.append(label_list[l])
            pred_row_labels.append(label_list[p])
    true_labels_list.append(true_row)
    pred_labels_list.append(pred_row_labels)

# Generate detailed report
print("\nDetailed classification report (per label):")
print(classification_report(true_labels_list, pred_labels_list, digits=4))

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: COMMA seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PERIOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SINGLE_QUOTE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: COLON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: HYPHEN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_l


Detailed classification report (per label):


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         ASH     0.0000    0.0000    0.0000         2
 EFT_BRACKET     0.0000    0.0000    0.0000        15
    EMICOLON     0.4000    0.2273    0.2899       132
      ERCENT     0.0000    0.0000    0.0000         0
       ERIOD     0.9624    0.9709    0.9666      7830
IGHT_BRACKET     0.0000    0.0000    0.0000        26
 INGLE_QUOTE     0.9957    0.9968    0.9962      3720
        LASH     1.0000    0.3333    0.5000         9
         LUS     0.0000    0.0000    0.0000         2
    MPERSAND     1.0000    1.0000    1.0000         8
      M_DASH     0.0000    0.0000    0.0000        27
       OLLAR     0.6000    0.3750    0.4615         8
        OLON     0.5596    0.5985    0.5784       259
        OMMA     0.7739    0.7936    0.7836      8460
 OUBLE_QUOTE     0.5376    0.3765    0.4429       247
       QUALS     0.0000    0.0000    0.0000         2
     UESTION     0.9061    0.9012    0.9037       739
  XCLAMATION     0.7500    

In [1]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer

# -------------------------------
# 1. Device setup
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------------------
# 2. Load tokenizer and model
# -------------------------------
tokenizer = AutoTokenizer.from_pretrained("thenlpresearcher/bert_punct_model")
model = AutoModelForTokenClassification.from_pretrained("thenlpresearcher/bert_punct_model")
model.to(device)

# -------------------------------
# 3. Prepare Trainer (only for prediction)
# -------------------------------
trainer = Trainer(
    model=model,
    tokenizer=tokenizer
)

# -------------------------------
# 4. Make predictions on test dataset
# -------------------------------
predictions, labels, _ = trainer.predict(test_dataset)  # test_dataset must be already prepared
pred_ids = np.argmax(predictions, axis=-1)

# -------------------------------
# 5. Map predictions and labels back to strings
# -------------------------------
true_labels_list = []
pred_labels_list = []

for label_row, pred_row in zip(labels, pred_ids):
    true_row = []
    pred_row_labels = []
    for l, p in zip(label_row, pred_row):
        if l != -100:  # ignore padding
            true_row.append(label_list[l])
            pred_row_labels.append(label_list[p])
    true_labels_list.append(true_row)
    pred_labels_list.append(pred_row_labels)


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-25 10:47:25.903069: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-25 10:47:25.955922: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-25 10:47:28.226987: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different n

Using device: cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 79.15 GiB of which 5.69 MiB is free. Process 35134 has 1.12 GiB memory in use. Process 66739 has 934.00 MiB memory in use. Process 21505 has 1.98 GiB memory in use. Process 18015 has 518.00 MiB memory in use. Process 74924 has 53.18 GiB memory in use. Process 113263 has 1.75 GiB memory in use. Process 44020 has 1.75 GiB memory in use. Process 70104 has 3.24 GiB memory in use. Process 58557 has 12.37 GiB memory in use. Process 95704 has 812.00 MiB memory in use. Process 63706 has 1.48 GiB memory in use. Of the allocated memory 1.07 GiB is allocated by PyTorch, and 2.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [23]:
from sklearn.metrics import classification_report

# Flatten lists for sklearn
y_true_flat = [label for seq in true_labels_list for label in seq]
y_pred_flat = [label for seq in pred_labels_list for label in seq]

print("\nDetailed classification report (per label) using sklearn:")
print(classification_report(y_true_flat, y_pred_flat, digits=4))


Detailed classification report (per label) using sklearn:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

               precision    recall  f1-score   support

    AMPERSAND     1.0000    1.0000    1.0000         8
        COLON     0.5710    0.6336    0.6006       292
        COMMA     0.7986    0.8220    0.8101      9916
       DOLLAR     0.6000    0.3750    0.4615         8
 DOUBLE_QUOTE     0.5837    0.3898    0.4674       313
      EM_DASH     0.0000    0.0000    0.0000        27
       EQUALS     0.0000    0.0000    0.0000         2
  EXCLAMATION     0.7500    0.1552    0.2571        58
         HASH     0.0000    0.0000    0.0000         2
       HYPHEN     0.6538    0.5157    0.5766      1117
 LEFT_BRACKET     0.0000    0.0000    0.0000        15
            O     0.9844    0.9858    0.9851    117294
      PERCENT     0.0000    0.0000    0.0000         0
       PERIOD     0.9662    0.9714    0.9688      8729
         PLUS     0.0000    0.0000    0.0000         2
     QUESTION     0.9108    0.8994    0.9051       795
RIGHT_BRACKET     0.0000    0.0000    0.0000        34
    SEMIC

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [33]:
punctuation_reverse_map = {v: k for k, v in punctuation_map.items()}
punctuation_reverse_map["O"] = ""   # no punctuation

In [30]:
import torch

def restore_punctuation(text, tokenizer, model, label_list, device=None):
    """
    text: raw unpunctuated string
    tokenizer: HF tokenizer
    model: punctuation restoration model (token classification)
    label_list: list mapping label IDs -> label strings
    """

    # Select device automatically if not provided
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Tokenize on CPU (normal), push tensors to GPU
    encoded = tokenizer(
        text.split(),            # token classification expects token-wise input
        is_split_into_words=True,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    encoded = {k: v.to(device) for k, v in encoded.items()}

    # Run model on GPU
    with torch.no_grad():
        logits = model(**encoded).logits
        predictions = torch.argmax(logits, dim=-1)[0].cpu().tolist()  
        # convert back to CPU to handle Python label mapping

    tokens = text.split()
    punctuated_tokens = []

    for token, pred_id in zip(tokens, predictions):
        label = label_list[pred_id]

        # No-punctuation label → just append token
        if label == "O":
            punctuated_tokens.append(token)
        else:
            # label looks like: COMMA, PERIOD, QUESTION
            punct_char = punctuation_reverse_map.get(label, "")
            punctuated_tokens.append(token + punct_char)

    return " ".join(punctuated_tokens)